In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append("../")


In [ ]:
import numpy as np
from iotools import simulate

nsamples = 200
ngenes   = 100
nsnps    = 1

expr = np.random.normal(0, 1, ngenes*nsamples).reshape(ngenes, nsamples)
snpinfo, gtnorm, gtcent = simulate.permuted_dosage(expr, nsnp = nsnps, fmin = 0.1, fmax = 0.9)


In [ ]:
# Get genotype variance as in C
import numpy as np
import ctypes
import os

def gt_var(geno):
    clib = np.ctypeslib.load_library('../lib/reverse_regression.so', ".")
    cgt_var = clib.genotype_variance
    cgt_var.restype = ctypes.c_bool
    cgt_var.argtypes = [np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED'),
                        ctypes.c_int,
                        ctypes.c_int,
                        np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED'),
                        ctypes.c_int                        
                       ]
    x = geno.reshape(-1,)
    nsnps = geno.shape[0]
    nsamples = geno.shape[1]
    SX2 = np.zeros(nsnps)
    success = cgt_var(x, nsnps, nsamples, SX2, 2)
    return SX2

In [ ]:
sigma_x2 = gt_var(gtnorm)

In [ ]:
U, s, VT = np.linalg.svd(expr.T, full_matrices=False)
print("U:", U.shape)
print("s:", s.shape)
print("VT:", VT.shape)

In [ ]:
sigmabeta = 0.01
sigma_x2 = np.var(gtnorm,axis=1) # ** 2
sb2 = sigmabeta ** 2 # np.repeat ???
k = min(nsamples, ngenes)
I = np.identity(k)
S = I*s

In [ ]:
# snpi = 0
# innerL = I*s*s + (sigma_x2[snpi]/sb2)*I
# innerL_inv = np.linalg.inv(innerL)
# V = np.transpose(VT)

# left_mat = np.matmul(V, innerL_inv)
# print("left_mat:", left_mat.shape)
# UT = np.transpose(U)

# StUt = np.matmul(S, UT)
# print("StUt:",StUt.shape) 

# A = np.matmul(left_mat, StUt)
# # B = np.matmul(A, X)
# print("A:",A.shape)

# Bi = np.matmul(A, gtnorm[1,:][np.newaxis].T)
# print(Bi[:10])

In [ ]:
# innerL = (I*s*s + (sigma_x2/sb2)*I)
# innerL_inv = np.linalg.inv(innerL)
# test = np.matmul(innerL_inv, S)
# test
snpi = 0
innerLinv_ST = (s / (s*s + (sigma_x2[snpi]/sb2))) * I
print("innerLinv_ST:", innerLinv_ST.shape)
UT = np.transpose(U)
V = np.transpose(VT)

innerLinv_STUT = np.matmul(innerLinv_ST, UT)
print("innerLinv_STUT:", innerLinv_STUT.shape)  # if G < N, then UT is k x N and then innerL_STUT is k x N
A2 = np.matmul(V, innerLinv_STUT)
print("A:", A2.shape)

Bi = np.matmul(A2, gtnorm[snpi,:][np.newaxis].T)
print(Bi[:10])

In [ ]:
innerLinv_ST

In [ ]:
import numpy as np
import ctypes
import os

def crevreg(geno, expr, sb2):
#     _path = os.path.dirname(".")
    clib = np.ctypeslib.load_library('../lib/reverse_regression.so', ".")
    cbetas = clib.betas
    cbetas.restype = ctypes.c_bool
    cbetas.argtypes = [np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED'),
                        np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED'),
                        np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED'),
                        ctypes.c_int,
                        ctypes.c_int,
                        ctypes.c_int,
                        np.ctypeslib.ndpointer(ctypes.c_double, ndim=1, flags='C_CONTIGUOUS, ALIGNED')
                       ]

    # print("expr shape is: genes x nsamples", expr.shape)
    # print("geno shape is: snps x nsamples", geno.shape)
    x = geno.reshape(-1,)
    y = expr.reshape(-1,)
    nsnps = geno.shape[0]
    nsamples = geno.shape[1]
    ngenes = expr.shape[0]
    B = np.zeros(ngenes * nsnps)
#     preB = np.zeros(ngenes * nsamples)
    success = cbetas(x, y, sb2, ngenes, nsnps, nsamples, B)
    return B


In [ ]:
# print(s)
# print(s*s)
# print(sigma_x2[snpi:snpi+3])
# print(sb2)
# print(innerLinv_STUT[0:5,0:5])
# print(innerLinv_ST[0:5,0:5])
# print(UT[0:5,0:5])
# print(A2.shape)
# print(A2)

# print("Bi: ", Bi.shape)
# print(Bi.reshape(-1,)[:100])

In [ ]:
sb2 = np.repeat(sigmabeta ** 2, gtnorm.shape[0])
B = crevreg(gtnorm, expr, sb2)

In [ ]:
B[:100]

In [ ]:
Bi.reshape(-1,)

In [ ]:
newB = B.reshape(nsnps, ngenes)

In [ ]:
newB[1]

In [ ]:
np.sort(newB[1])

In [ ]:
np.argpartition(newB[1], 10)

In [ ]:
a = np.zeros((3,4))
b = np.ones((3,2))
ind = np.array([0,3])
inv_ind = np.delete(np.arange(a.shape[1]), ind)
a[:,inv_ind] = b
a

In [ ]:
allbeta = np.array([])
allbeta = np.vstack((a, a ))